In [16]:
import pandas as pd
import numpy as np
import xlsxwriter
from openpyxl import load_workbook
from math import log10, floor
import sys
sys._enablelegacywindowsfsencoding()


def round_to_1(x):
    return round(x, -int(floor(log10(x))))
def round_to_reference(x, y):
    return round(x, -int(floor(log10(y))))

eps = 'ε'
Q = 'Q'
G = 'Γ'
I = 'I'
delta = 'δ'
tetta = 'θ'
phi = 'φ'
H = 'Hn'
Hmax = 'Hnmax'

def get_ready_data(name):
    data = pd.read_csv(name, header=0, sep=";")
    data = data.drop(data.columns[2:22], axis=1)
    return data



def find_param(param, print_info = 0):
    param_value = []
    param_error = []
    param_total = []
    param_model = []
    i=0
    for column in data.columns:
        if (param in column and not('Δ' in column) and (not('†' in column))):
            #print data[column]
                param_value.append(data[column][0])
                param_model.append(column)
        if ('Δ'+param in column):
            #if ('Δ' in column) and (not('†' in column)):
            if (data[column][0] < 0.015) and (data[column][0] != 0):
                param_error.append(0.01)
            elif data[column][0] == 0:
                param_error.append(0.0000001)
            else:
                param_error.append(data[column][0])
        if ('†Δ'+param in column):
            #if ('Δ' in column) and (not('†' in column)):
            if (data[column][0] < 0.015) and (data[column][0] != 0):
                param_error[-1]=(0.01)
            elif data[column][0] == 0:
                param_error[-1]=(0.0000001)
            else:
                param_error[-1]=(data[column][0])

    error_r = list(map(round_to_1, param_error))
    param_r = list(map(round_to_reference, param_value, error_r))

    
    for i in range(0, len(param_r)):
        if (error_r[i] < 0.000001):
            param_r[i] = round(param_r[i], 1)
            param_total.append(str(param_r[i]))
        else:
            param_total.append(str(param_r[i])+'±'+str(error_r[i]))
            
        
    if print_info == 1:
        print (len(param_total), len(param_total)-len(param_model), param_total)
        print (param_model)
    if not param_total:
        param_total = [' ', ' ', ' ']
        return param_total
    else:
        return param_total

import os

#files = []
#for i in os.listdir("."):
#    if i.endswith('.csv'):
#        files.append(open(i))
        
#files_spc = []
#for i in os.listdir("."):
#    if i.endswith('.spc'):
#        files_spc.append(open(i))

path = r'C:/Users/nikvo/Google Drive/VostrovMossb/2018_article/Combined RT+He/relax/'
path2 = r'C:/Users/Сергей/Google Диск/VostrovMossb/2018_article/Combined RT+He/relax/'
path = path2

def folder(str1):
    files = []
    for i in os.listdir(path+'01 and 03/'+str1):
        if i.endswith('.csv'):
            files.append(open(path+'01 and 03/'+str1+i))
        
    files_spc = []
    for i in os.listdir(path+'01 and 03/'+str1):
        if i.endswith('.spc'):
            files_spc.append(open(path+'01 and 03/'+str1+i))
    return files, files_spc


def find_name(str_n, f_array):
    for f in f_array:
        if str_n in f.name:
            return f.name
#print(data.T.to_string())

def give_links(table, n, path, f_spc_array):
    for f in f_spc_array:
        if n[0:5] in f.name:
            table.loc[charge, '4Fe3+']['data', 'link'] = 'file:///'+path+f.name
            print (path+f.name)
 
    table.loc[charge, 'MFe3+']['data', 'link'] = 'file:///'+path+n[0:-3]+'mdl'
    table.loc[charge, '4Fe2+']['data', 'link'] = 'file:///'+path+n[0:-3]+'html'
    table.loc[charge, '4Fe3+']['data', 'data'] = 'spc'
    table.loc[charge, 'MFe3+']['data', 'data'] = 'mdl'
    table.loc[charge, '4Fe2+']['data', 'data'] = 'html'
    
def give_links_zn(table, n, path, f_spc_array):
    for f in f_spc_array:
        if n[0:5] in f.name:
            table.loc[charge, '4Fe3+']['data', 'link'] = path+f.name
 
    table.loc[charge, 'RelaxFe3+']['data', 'link'] = path+n[0:-3]+'mdl'
    table.loc[charge, '4Fe2+']['data', 'link'] = path+n[0:-3]+'html'
    table.loc[charge, '4Fe3+']['data', 'data'] = 'spc'
    table.loc[charge, 'RelaxFe3+']['data', 'data'] = 'mdl'
    table.loc[charge, '4Fe2+']['data', 'data'] = 'html'
    
#n = find_name('Co0.2_IN', files)

#file:///



In [17]:
table = pd.DataFrame(columns = ['Intensity', delta, eps, ])
oney = ['He', 'He', 'He', 'He', 'He', 'He', 'He', 'RT', 'RT', 'RT', 'RT', 'RT', 'data', 'data']
twoy = [I, delta, 'e^2qQ', H, tetta, phi, G, I, delta, eps, G, Q, 'data', 'link']
spectra = ['1', '0.5', '0']
partial_sp = ['4Fe3+', 'MFe3+', '4Fe2+', '.', "RelaxFe2+", '..']
onex = []
twox = []
for i in range(0, len(spectra)):
    twox.append(partial_sp)
    for j in range(0, len(partial_sp)):
        onex.append(spectra[i])
twox = [j for sub in twox for j in sub]
arraysy = [oney, twoy]
arraysx = [onex, twox]
table = pd.DataFrame(columns=arraysy, index = arraysx).fillna(' ')
charge = spectra[0]
 
files, files_spc = folder('co01x1/')
n = find_name('.', files)
data = get_ready_data(n)
give_links(table, n, path, files_spc)  


table.loc[charge, '.']['He', tetta] = 'ln(R)'
table.loc[charge, '.']['He', phi] = 'α'
table.loc[charge, '.']['He', 'e^2qQ'] = eps

    
table.loc[charge, '4Fe3+']['He', 'I'] = find_param('I1')[1]
table.loc[charge, 'MFe3+']['He', 'I'] = find_param('I2')[0]
table.loc[charge, '4Fe2+']['He', 'I'] = find_param('I3')[1]
table.loc[charge, 'RelaxFe2+']['He', 'I'] = find_param('I4')[0]
    
table.loc[charge, '4Fe3+']['He', delta] = find_param(delta+' / 1')[0]
table.loc[charge, 'MFe3+']['He', delta] = find_param(delta+' / 2')[0]
table.loc[charge, '4Fe2+']['He', delta] = find_param(delta+' / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', delta] = find_param(delta+' / 4')[0]
   
table.loc[charge, '4Fe3+']['He', 'e^2qQ'] = find_param(Q+' / 1')[0]
table.loc[charge, 'MFe3+']['He', 'e^2qQ'] = find_param(Q+' / 2')[0]
table.loc[charge, '4Fe2+']['He', 'e^2qQ'] = find_param(Q+' / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', 'e^2qQ'] = find_param(eps)[0]
    
table.loc[charge, '4Fe3+']['He', H] = find_param(H+' / 1')[0]
table.loc[charge, 'MFe3+']['He', H] = find_param(Hmax+' / 2')[0]
table.loc[charge, '4Fe2+']['He', H] = find_param(H+' / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', H] = find_param('H / 4')[0]
    
table.loc[charge, '4Fe3+']['He', tetta] = find_param(tetta+' / 1')[0]
table.loc[charge, 'MFe3+']['He', tetta] = find_param(tetta+'max / 2')[0]
table.loc[charge, '4Fe2+']['He', tetta] = find_param(tetta+' / 3')[0]

table.loc[charge, '4Fe3+']['He', phi] = find_param(phi+' / 1')[0]
table.loc[charge, 'MFe3+']['He', phi] = find_param(phi+'max / 2')[0]
table.loc[charge, '4Fe2+']['He', phi] = find_param(phi+' / 3')[0]
   
table.loc[charge, '4Fe3+']['He', G] = find_param(G+' / 1')[0]    
table.loc[charge, 'MFe3+']['He', G] = find_param(G+' / 2')[0]
table.loc[charge, '4Fe2+']['He', G] = find_param('Γ₁ / 3 /')[0]+' '+find_param('Γ₂ / 3')[0]+' '+find_param('Γ₃ / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', G] = find_param(G+' / 4')[0]

table.loc[charge, 'RelaxFe2+']['He', tetta] = find_param('ln(R)')[0]
table.loc[charge, 'RelaxFe2+']['He', phi] = find_param('α')[3]
table.loc[charge, 'RelaxFe2+']['He', 'e^2qQ'] = find_param(eps)[0]



table.loc[charge, '4Fe3+']['RT', 'I'] = find_param('I5')[0]
table.loc[charge, 'MFe3+']['RT', 'I'] = find_param('I6')[0]
table.loc[charge, '4Fe2+']['RT', 'I'] = find_param('I7')[0]
table.loc[charge, 'RelaxFe2+']['RT', 'I'] = find_param('I8')[0]

table.loc[charge, '4Fe3+']['RT', delta] = find_param('delta5')[0]
table.loc[charge, 'MFe3+']['RT', delta] = find_param('delta6')[0]
table.loc[charge, '4Fe2+']['RT', delta] = find_param('delta7')[0]
table.loc[charge, 'RelaxFe2+']['RT', delta] = find_param('delta8')[0]

table.loc[charge, '4Fe3+']['RT', eps] = find_param('eps5')[0]
table.loc[charge, 'MFe3+']['RT', eps] = find_param('eps6')[0]
table.loc[charge, '4Fe2+']['RT', eps] = find_param('eps7')[0]
table.loc[charge, 'RelaxFe2+']['RT', eps] = find_param('eps8')[0]

table.loc[charge, '4Fe3+']['RT', G] = find_param('Г5')[0]
table.loc[charge, 'MFe3+']['RT', G] = find_param('Г6')[0]
table.loc[charge, '4Fe2+']['RT', G] = find_param('Г7')[0]
table.loc[charge, 'RelaxFe2+']['RT', G] = find_param('Г8')[0]



table


C:/Users/Сергей/Google Диск/VostrovMossb/2018_article/Combined RT+He/relax/C:/Users/Сергей/Google Диск/VostrovMossb/2018_article/Combined RT+He/relax/01 and 03/co01x1/12154+14069_1024.spc


He                                                  \
                        I          δ      e^2qQ           Hn      θ     φ   
1   4Fe3+             0.0        0.5       -2.5        501.6   34.3  72.2   
    MFe3+             0.0        0.5                                        
    4Fe2+      88.36±0.01  1.35±0.01  5.56±0.01  123.23±0.08    0.0   0.0   
    .                                         ε               ln(R)     α   
    RelaxFe2+    12.0±2.0  1.35±0.01  1.71±0.05  115.95±0.03    0.0   3.1   
    ..                                                                      
0.5 4Fe3+                                                                   
    MFe3+                                                                   
    4Fe2+                                                                   
    .                                                                       
    RelaxFe2+                                                               
    ..                                                                      
0   4Fe3+                                                                   
    MFe3+                                                                   
    4Fe2+                                                                   
    .                                                                       
    RelaxFe2+                                                               
    ..                                                                      

                                                   RT                        \
                                          Γ         I          δ          ε   
1   4Fe3+                               3.3       0.0        0.4        0.7   
    MFe3+                               0.7   6.8±0.4  0.44±0.01  0.45±0.01   
    4Fe2+      0.33±0.01 0.3±0.01 0.28±0.01  82.0±2.0  1.23±0.01  1.49±0.01   
    .                                                                         
    RelaxFe2+                           0.8  11.0±2.0  1.24±0.01  1.41±0.02   
    ..                                                                        
0.5 4Fe3+                                                                     
    MFe3+                                                                     
    4Fe2+                                                                     
    .                                                                         
    RelaxFe2+                                                                 
    ..                                                                        
0   4Fe3+                                                                     
    MFe3+                                                                     
    4Fe2+                                                                     
    .                                                                         
    RelaxFe2+                                                                 
    ..                                                                        

                             data  \
                       Γ  Q  data   
1   4Fe3+            0.9      spc   
    MFe3+       0.6±0.02      mdl   
    4Fe2+      0.25±0.01     html   
    .                               
    RelaxFe2+  0.73±0.08            
    ..                              
0.5 4Fe3+                           
    MFe3+                           
    4Fe2+                           
    .                               
    RelaxFe2+                       
    ..                              
0   4Fe3+                           
    MFe3+                           
    4Fe2+                           
    .                               
    RelaxFe2+                       
    ..                              

                                                                  
                                                            link  
1   4Fe3+      file:///C:/

In [18]:
  
charge = spectra[1]
files, files_spc = folder('co01x058/')    
n = find_name('.', files)
data = get_ready_data(n)
give_links(table, n, path, files_spc)  


table.loc[charge, '.']['He', tetta] = 'ln(R)'
table.loc[charge, '.']['He', phi] = 'α'
table.loc[charge, '.']['He', 'e^2qQ'] = eps

    
table.loc[charge, '4Fe3+']['He', 'I'] = find_param('I1')[1]
table.loc[charge, 'MFe3+']['He', 'I'] = find_param('I2')[0]
table.loc[charge, '4Fe2+']['He', 'I'] = find_param('I3')[1]
table.loc[charge, 'RelaxFe2+']['He', 'I'] = find_param('I4')[0]
    
table.loc[charge, '4Fe3+']['He', delta] = find_param(delta+' / 1')[0]
table.loc[charge, 'MFe3+']['He', delta] = find_param(delta+' / 2')[0]
table.loc[charge, '4Fe2+']['He', delta] = find_param(delta+' / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', delta] = find_param(delta+' / 4')[0]
   
table.loc[charge, '4Fe3+']['He', 'e^2qQ'] = find_param(Q+' / 1')[0]
table.loc[charge, 'MFe3+']['He', 'e^2qQ'] = find_param(Q+' / 2')[0]
table.loc[charge, '4Fe2+']['He', 'e^2qQ'] = find_param(Q+' / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', 'e^2qQ'] = find_param(eps)[0]
    
table.loc[charge, '4Fe3+']['He', H] = find_param(H+' / 1')[0]
table.loc[charge, 'MFe3+']['He', H] = find_param(Hmax+' / 2')[0]
table.loc[charge, '4Fe2+']['He', H] = find_param(H+' / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', H] = find_param('H / 4')[0]
    
table.loc[charge, '4Fe3+']['He', tetta] = find_param(tetta+' / 1')[0]
table.loc[charge, 'MFe3+']['He', tetta] = find_param(tetta+'max / 2')[0]
table.loc[charge, '4Fe2+']['He', tetta] = find_param(tetta+' / 3')[0]

table.loc[charge, '4Fe3+']['He', phi] = find_param(phi+' / 1')[0]
table.loc[charge, 'MFe3+']['He', phi] = find_param(phi+'max / 2')[0]
table.loc[charge, '4Fe2+']['He', phi] = find_param(phi+' / 3')[0]
   
table.loc[charge, '4Fe3+']['He', G] = find_param(G+' / 1')[0]    
table.loc[charge, 'MFe3+']['He', G] = find_param(G+' / 2')[0]
table.loc[charge, '4Fe2+']['He', G] = find_param('Γ₁ / 3 /')[0]+' '+find_param('Γ₂ / 3')[0]+' '+find_param('Γ₃ / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', G] = find_param(G+' / 4')[0]

table.loc[charge, 'RelaxFe2+']['He', tetta] = find_param('ln(R)')[0]
table.loc[charge, 'RelaxFe2+']['He', phi] = find_param('α')[3]
table.loc[charge, 'RelaxFe2+']['He', 'e^2qQ'] = find_param(eps)[0]



table.loc[charge, '4Fe3+']['RT', 'I'] = find_param('I5')[0]
table.loc[charge, 'MFe3+']['RT', 'I'] = find_param('I6')[0]
table.loc[charge, '4Fe2+']['RT', 'I'] = find_param('I7')[0]
table.loc[charge, 'RelaxFe2+']['RT', 'I'] = find_param('I8')[0]

table.loc[charge, '4Fe3+']['RT', delta] = find_param('delta5')[0]
table.loc[charge, 'MFe3+']['RT', delta] = find_param('delta6')[0]
table.loc[charge, '4Fe2+']['RT', delta] = find_param('delta7')[0]
table.loc[charge, 'RelaxFe2+']['RT', delta] = find_param('delta8')[0]

table.loc[charge, '4Fe3+']['RT', eps] = find_param('eps5')[0]
table.loc[charge, 'MFe3+']['RT', eps] = find_param('eps6')[0]
table.loc[charge, '4Fe2+']['RT', eps] = find_param('eps7')[0]
table.loc[charge, 'RelaxFe2+']['RT', eps] = find_param('eps8')[0]

table.loc[charge, '4Fe3+']['RT', G] = find_param('Г5')[0]
table.loc[charge, 'MFe3+']['RT', G] = find_param('Г6')[0]
table.loc[charge, '4Fe2+']['RT', G] = find_param('Г7')[0]
table.loc[charge, 'RelaxFe2+']['RT', G] = find_param('Г8')[0]



table


C:/Users/Сергей/Google Диск/VostrovMossb/2018_article/Combined RT+He/relax/C:/Users/Сергей/Google Диск/VostrovMossb/2018_article/Combined RT+He/relax/01 and 03/co01x058/12160+161+162+14080+81_1024.spc


He                                                \
                        I          δ       e^2qQ           Hn         θ   
1   4Fe3+             0.0        0.5        -2.5        501.6      34.3   
    MFe3+             0.0        0.5                                      
    4Fe2+      88.36±0.01  1.35±0.01   5.56±0.01  123.23±0.08       0.0   
    .                                          ε                  ln(R)   
    RelaxFe2+    12.0±2.0  1.35±0.01   1.71±0.05  115.95±0.03       0.0   
    ..                                                                    
0.5 4Fe3+      12.87±0.01  0.53±0.01  -2.63±0.01    500.5±0.6  33.5±0.3   
    MFe3+        17.0±1.0  0.58±0.01  -2.12±0.03    516.0±1.0  66.8±0.6   
    4Fe2+      53.43±0.01  1.35±0.01   5.55±0.01  123.09±0.09       0.0   
    .                                          ε                  ln(R)   
    RelaxFe2+    16.7±0.7        1.4   1.64±0.02    120.0±3.0       0.0   
    ..                                                                    
0   4Fe3+                                                                 
    MFe3+                                                                 
    4Fe2+                                                                 
    .                                                                     
    RelaxFe2+                                                             
    ..                                                                    

                                                                RT             \
                        φ                              Γ         I          δ   
1   4Fe3+            72.2                            3.3       0.0        0.4   
    MFe3+                                            0.7   6.8±0.4  0.44±0.01   
    4Fe2+             0.0   0.33±0.01 0.3±0.01 0.28±0.01  82.0±2.0  1.23±0.01   
    .                   α                                                       
    RelaxFe2+         3.1                            0.8  11.0±2.0  1.24±0.01   
    ..                                                                          
0.5 4Fe3+      85.39±0.02                      0.32±0.02  15.0±1.0  0.42±0.01   
    MFe3+             0.0                      0.44±0.02  20.0±1.0  0.43±0.01   
    4Fe2+             0.0  0.32±0.01 0.29±0.01 0.28±0.01  49.5±0.6  1.23±0.01   
    .                   α                                                       
    RelaxFe2+         4.0                            0.8  15.5±0.7  1.35±0.01   
    ..                                                                          
0   4Fe3+                                                                       
    MFe3+                                                                       
    4Fe2+                                                                       
    .                                                                           
    RelaxFe2+                                                                   
    ..                                                                          

                                        data  \
                       ε          Γ  Q  data   
1   4Fe3+            0.7        0.9      spc   
    MFe3+      0.45±0.01   0.6±0.02      mdl   
    4Fe2+      1.49±0.01  0.25±0.01     html   
    .                                          
    RelaxFe2+  1.41±0.02  0.73±0.08            
    ..                                         
0.5 4Fe3+      0.76±0.01  0.27±0.01      spc   
    MFe3+      0.61±0.01  0.44±0.02      mdl   
    4Fe2+      1.48±0.01  0.25±0.01     html   
    .                                          
    RelaxFe2+   1.3±0.01        0.8            
    ..                                         
0   4Fe3+                                      
    MFe3+                                      
    4Fe2+                                      
    .                                          
    RelaxFe2+                        

In [19]:
  
charge = spectra[2]
files, files_spc = folder('co01x01/')
n = find_name('', files)
data = get_ready_data(n)
give_links(table, n, path, files_spc)  

table.loc[charge, '.']['He', tetta] = 'ln(R)'
table.loc[charge, '.']['He', phi] = 'α'
table.loc[charge, '.']['He', 'e^2qQ'] = eps

    
table.loc[charge, '4Fe3+']['He', 'I'] = find_param('I1')[1]
table.loc[charge, 'MFe3+']['He', 'I'] = find_param('I2')[0]
table.loc[charge, '4Fe2+']['He', 'I'] = find_param('I3')[1]
table.loc[charge, 'RelaxFe2+']['He', 'I'] = find_param('I4')[0]
    
table.loc[charge, '4Fe3+']['He', delta] = find_param(delta+' / 1')[0]
table.loc[charge, 'MFe3+']['He', delta] = find_param(delta+' / 2')[0]
table.loc[charge, '4Fe2+']['He', delta] = find_param(delta+' / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', delta] = find_param(delta+' / 4')[0]
   
table.loc[charge, '4Fe3+']['He', 'e^2qQ'] = find_param(Q+' / 1')[0]
table.loc[charge, 'MFe3+']['He', 'e^2qQ'] = find_param(Q+' / 2')[0]
table.loc[charge, '4Fe2+']['He', 'e^2qQ'] = find_param(Q+' / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', 'e^2qQ'] = find_param(eps)[0]
    
table.loc[charge, '4Fe3+']['He', H] = find_param(H+' / 1')[0]
table.loc[charge, 'MFe3+']['He', H] = find_param(Hmax+' / 2')[0]
table.loc[charge, '4Fe2+']['He', H] = find_param(H+' / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', H] = find_param('H / 4')[0]
    
table.loc[charge, '4Fe3+']['He', tetta] = find_param(tetta+' / 1')[0]
table.loc[charge, 'MFe3+']['He', tetta] = find_param(tetta+'max / 2')[0]
table.loc[charge, '4Fe2+']['He', tetta] = find_param(tetta+' / 3')[0]

table.loc[charge, '4Fe3+']['He', phi] = find_param(phi+' / 1')[0]
table.loc[charge, 'MFe3+']['He', phi] = find_param(phi+'max / 2')[0]
table.loc[charge, '4Fe2+']['He', phi] = find_param(phi+' / 3')[0]
   
table.loc[charge, '4Fe3+']['He', G] = find_param(G+' / 1')[0]    
table.loc[charge, 'MFe3+']['He', G] = find_param(G+' / 2')[0]
table.loc[charge, '4Fe2+']['He', G] = find_param('Γ₁ / 3 /')[0]+' '+find_param('Γ₂ / 3')[0]+' '+find_param('Γ₃ / 3')[0]
table.loc[charge, 'RelaxFe2+']['He', G] = find_param(G+' / 4')[0]

table.loc[charge, 'RelaxFe2+']['He', tetta] = find_param('ln(R)')[0]
table.loc[charge, 'RelaxFe2+']['He', phi] = find_param('α')[3]
table.loc[charge, 'RelaxFe2+']['He', 'e^2qQ'] = find_param(eps)[0]



table.loc[charge, '4Fe3+']['RT', 'I'] = find_param('I5')[0]
table.loc[charge, 'MFe3+']['RT', 'I'] = find_param('I6')[0]
table.loc[charge, '4Fe2+']['RT', 'I'] = find_param('I7')[0]
table.loc[charge, 'RelaxFe2+']['RT', 'I'] = find_param('I8')[0]

table.loc[charge, '4Fe3+']['RT', delta] = find_param('delta5')[0]
table.loc[charge, 'MFe3+']['RT', delta] = find_param('delta6')[0]
table.loc[charge, '4Fe2+']['RT', delta] = find_param('delta7')[0]
table.loc[charge, 'RelaxFe2+']['RT', delta] = find_param('delta8')[0]

table.loc[charge, '4Fe3+']['RT', eps] = find_param('eps5')[0]
table.loc[charge, 'MFe3+']['RT', eps] = find_param('eps6')[0]
table.loc[charge, '4Fe2+']['RT', eps] = find_param('eps7')[0]
table.loc[charge, 'RelaxFe2+']['RT', eps] = find_param('eps8')[0]

table.loc[charge, '4Fe3+']['RT', G] = find_param('Г5')[0]
table.loc[charge, 'MFe3+']['RT', G] = find_param('Г6')[0]
table.loc[charge, '4Fe2+']['RT', G] = find_param('Г7')[0]
table.loc[charge, 'RelaxFe2+']['RT', G] = find_param('Г8')[0]



table

C:/Users/Сергей/Google Диск/VostrovMossb/2018_article/Combined RT+He/relax/C:/Users/Сергей/Google Диск/VostrovMossb/2018_article/Combined RT+He/relax/01 and 03/co01x01/12158+14019_1024.spc


He                                                \
                        I          δ       e^2qQ           Hn         θ   
1   4Fe3+             0.0        0.5        -2.5        501.6      34.3   
    MFe3+             0.0        0.5                                      
    4Fe2+      88.36±0.01  1.35±0.01   5.56±0.01  123.23±0.08       0.0   
    .                                          ε                  ln(R)   
    RelaxFe2+    12.0±2.0  1.35±0.01   1.71±0.05  115.95±0.03       0.0   
    ..                                                                    
0.5 4Fe3+      12.87±0.01  0.53±0.01  -2.63±0.01    500.5±0.6  33.5±0.3   
    MFe3+        17.0±1.0  0.58±0.01  -2.12±0.03    516.0±1.0  66.8±0.6   
    4Fe2+      53.43±0.01  1.35±0.01   5.55±0.01  123.09±0.09       0.0   
    .                                          ε                  ln(R)   
    RelaxFe2+    16.7±0.7        1.4   1.64±0.02    120.0±3.0       0.0   
    ..                                                                    
0   4Fe3+      37.54±0.01  0.53±0.01  -2.57±0.01    499.7±0.2  33.1±0.1   
    MFe3+        51.7±0.7  0.57±0.01  -2.16±0.01    513.0±0.7  60.3±0.2   
    4Fe2+       3.39±0.01        1.3         5.6  154.51±0.01       0.0   
    .                                          ε                  ln(R)   
    RelaxFe2+     7.4±0.5        1.4         1.6  107.61±0.02       0.0   
    ..                                                                    

                                                                RT             \
                        φ                              Γ         I          δ   
1   4Fe3+            72.2                            3.3       0.0        0.4   
    MFe3+                                            0.7   6.8±0.4  0.44±0.01   
    4Fe2+             0.0   0.33±0.01 0.3±0.01 0.28±0.01  82.0±2.0  1.23±0.01   
    .                   α                                                       
    RelaxFe2+         3.1                            0.8  11.0±2.0  1.24±0.01   
    ..                                                                          
0.5 4Fe3+      85.39±0.02                      0.32±0.02  15.0±1.0  0.42±0.01   
    MFe3+             0.0                      0.44±0.02  20.0±1.0  0.43±0.01   
    4Fe2+             0.0  0.32±0.01 0.29±0.01 0.28±0.01  49.5±0.6  1.23±0.01   
    .                   α                                                       
    RelaxFe2+         4.0                            0.8  15.5±0.7  1.35±0.01   
    ..                                                                          
0   4Fe3+      83.29±0.01                      0.37±0.01  38.0±1.0  0.43±0.01   
    MFe3+            20.0                      0.44±0.01  52.0±1.0  0.44±0.01   
    4Fe2+             0.0                    0.6 0.6 0.6   3.0±0.4  1.17±0.02   
    .                   α                                                       
    RelaxFe2+         0.4                            0.8   6.5±0.5  1.19±0.02   
    ..                                                                          

                                        data  \
                       ε          Γ  Q  data   
1   4Fe3+            0.7        0.9      spc   
    MFe3+      0.45±0.01   0.6±0.02      mdl   
    4Fe2+      1.49±0.01  0.25±0.01     html   
    .                                          
    RelaxFe2+  1.41±0.02  0.73±0.08            
    ..                                         
0.5 4Fe3+      0.76±0.01  0.27±0.01      spc   
    MFe3+      0.61±0.01  0.44±0.02      mdl   
    4Fe2+      1.48±0.01  0.25±0.01     html   
    .                                          
    RelaxFe2+   1.3±0.01        0.8            
    ..                                         
0   4Fe3+      0.74±0.01  0.27±0.01      spc   
    MFe3+      0.62±0.01  0.44±0.01      mdl   
    4Fe2+       1.5±0.02  0.31±0.03     html   
    .                                          
    RelaxFe2+  1.05±0.03  1.01±0.06  

In [20]:
table

He                                                \
                        I          δ       e^2qQ           Hn         θ   
1   4Fe3+             0.0        0.5        -2.5        501.6      34.3   
    MFe3+             0.0        0.5                                      
    4Fe2+      88.36±0.01  1.35±0.01   5.56±0.01  123.23±0.08       0.0   
    .                                          ε                  ln(R)   
    RelaxFe2+    12.0±2.0  1.35±0.01   1.71±0.05  115.95±0.03       0.0   
    ..                                                                    
0.5 4Fe3+      12.87±0.01  0.53±0.01  -2.63±0.01    500.5±0.6  33.5±0.3   
    MFe3+        17.0±1.0  0.58±0.01  -2.12±0.03    516.0±1.0  66.8±0.6   
    4Fe2+      53.43±0.01  1.35±0.01   5.55±0.01  123.09±0.09       0.0   
    .                                          ε                  ln(R)   
    RelaxFe2+    16.7±0.7        1.4   1.64±0.02    120.0±3.0       0.0   
    ..                                                                    
0   4Fe3+      37.54±0.01  0.53±0.01  -2.57±0.01    499.7±0.2  33.1±0.1   
    MFe3+        51.7±0.7  0.57±0.01  -2.16±0.01    513.0±0.7  60.3±0.2   
    4Fe2+       3.39±0.01        1.3         5.6  154.51±0.01       0.0   
    .                                          ε                  ln(R)   
    RelaxFe2+     7.4±0.5        1.4         1.6  107.61±0.02       0.0   
    ..                                                                    

                                                                RT             \
                        φ                              Γ         I          δ   
1   4Fe3+            72.2                            3.3       0.0        0.4   
    MFe3+                                            0.7   6.8±0.4  0.44±0.01   
    4Fe2+             0.0   0.33±0.01 0.3±0.01 0.28±0.01  82.0±2.0  1.23±0.01   
    .                   α                                                       
    RelaxFe2+         3.1                            0.8  11.0±2.0  1.24±0.01   
    ..                                                                          
0.5 4Fe3+      85.39±0.02                      0.32±0.02  15.0±1.0  0.42±0.01   
    MFe3+             0.0                      0.44±0.02  20.0±1.0  0.43±0.01   
    4Fe2+             0.0  0.32±0.01 0.29±0.01 0.28±0.01  49.5±0.6  1.23±0.01   
    .                   α                                                       
    RelaxFe2+         4.0                            0.8  15.5±0.7  1.35±0.01   
    ..                                                                          
0   4Fe3+      83.29±0.01                      0.37±0.01  38.0±1.0  0.43±0.01   
    MFe3+            20.0                      0.44±0.01  52.0±1.0  0.44±0.01   
    4Fe2+             0.0                    0.6 0.6 0.6   3.0±0.4  1.17±0.02   
    .                   α                                                       
    RelaxFe2+         0.4                            0.8   6.5±0.5  1.19±0.02   
    ..                                                                          

                                        data  \
                       ε          Γ  Q  data   
1   4Fe3+            0.7        0.9      spc   
    MFe3+      0.45±0.01   0.6±0.02      mdl   
    4Fe2+      1.49±0.01  0.25±0.01     html   
    .                                          
    RelaxFe2+  1.41±0.02  0.73±0.08            
    ..                                         
0.5 4Fe3+      0.76±0.01  0.27±0.01      spc   
    MFe3+      0.61±0.01  0.44±0.02      mdl   
    4Fe2+      1.48±0.01  0.25±0.01     html   
    .                                          
    RelaxFe2+   1.3±0.01        0.8            
    ..                                         
0   4Fe3+      0.74±0.01  0.27±0.01      spc   
    MFe3+      0.62±0.01  0.44±0.01      mdl   
    4Fe2+       1.5±0.02  0.31±0.03     html   
    .                                          
    RelaxFe2+  1.05±0.03  1.01±0.06  

In [15]:
writer = pd.ExcelWriter(r"tableCo01.xlsx", engine = 'openpyxl')
table.to_excel(writer, sheet_name='Mn')
writer.save()
#writer.close()